## Data processing from raw CPCSSN data

<b>Starting raw dataset:</b>
<ul>
    <li>raw dataframe extracted from the 2000-2015 portion of the CPCSSN database (multiple records for each patient) </li>
    <li>Exclusion criteria:  age <18 years at the time of the first primary care encounter, diagnosis of Type I diabetes or gestational diabetes </li>
    <li> for each primary care encounter, inclusion of patient's age, sex at birth, routinely extracted blood biomarkers (Fasting Plasma Glucose, High-Density Lipoproteins,Low-Density Lipoproteins, Triglycerides, Total Cholesterol), blood pressure measurements (systolic, diastolic), body mass index (BMI), prescription of specific groups of medications (antidepressants, corticosteroids, antihypertensives, cholesterol lowering medications, quit-smoking
medications),smoking (non-smoker, former-smoker, current-smoker), and diagnosis of specific comorbidities coded in CPCSSN DiseaseCase table with related diagnosis date (hypertension, chronic obstructive pulmonary disease, depression osteoarthritis) </li>
   <li> Inclusion of patients information up to six months prior to the diagnosis of T2D ( for patients who will develop T2D before 2015, 'T2D'=1), or information up to the last encounter available before the end of the database observation period (for CPCSSN patients who will not develop T2D before 2015, 'T2D'=0)</li>    
<li>0 female, 1 male</li>
    
</ul>

In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [ ]:
starting_dataset=pd.read_csv('pathname', delimiter=',')
starting_dataset.columns 
'''{'Patient_ID', 'D', 'age', 'sex', 'sbp', 'dbp', 'BMI', 'LDL', 'HDL',
       'TG', 'FBS', 'Total_Cholesterol', 'Depression', 'Depression_OnsetDate',
       'HTN', 'HTN_OnsetDate', 'OA', 'OA_OnsetDate', 'COPD', 'COPD_Date',
       'T2D', 'T2D_OnsetDate', 'antidepressant',
       'cholesterol_lowering_meds', 'antihypertensive_meds', 'corticosteroids',
       'smoking_meds'}'''

In [ ]:
starting_dataset.dtypes

## Extract the most recent record for each future T2D patient ('T2D'=1)

In [ ]:
#Extract records of future T2D patients only
T2DM=starting_dataset.loc[starting_dataset['T2D']==1]
T2DM

In [ ]:
#check number of unique subjects
T2DM['Patient_ID'].nunique()

In [ ]:
#Sort records by EncounterDate (D)
sorted_T2DM = T2DM.sort_values(by='D',ascending=False)
sorted_T2DM

In [ ]:
#select the most recent encounter for each patient
T2DM_unique = sorted_T2DM.drop_duplicates('Patient_ID', keep='first')
T2DM_unique

Once, the most recent encounter for each patient has been extracted, we define the <b>primary observation window</b> as the temporal window centred in 'Encounter Date' (D) +- 6 months

In [ ]:
''' Evaluate presence of comorbidities before the start of the primary observation window to ensure temporal ordering between possible onset
of comorbidities and measured biomarkers.'''
#compute temporal distance between 'Encounter date' and 'comorbidity onset date' 
T2DM_unique.loc[:,'D_COPD_diff']=T2DM_unique.loc[:,'D']-T2DM_unique.loc[:,'COPD_Date']
T2DM_unique.loc[:,'D_OA_diff']=T2DM_unique.loc[:,'D']-T2DM_unique.loc[:,'OA_OnsetDate']
T2DM_unique.loc[:,'D_HTN_diff']=T2DM_unique.loc[:,'D']-T2DM_unique.loc[:,'HTN_OnsetDate']
T2DM_unique.loc[:,'D_Depression_diff']=T2DM_unique.loc[:,'D']-T2DM_unique.loc[:,'Depression_OnsetDate']

In [ ]:
#comorbidity =1 if the comorbidity onset occurred before the primary observation window
#comorbidity= 0 if the comorbidity onset never occurred or occurred after the primary observation window
#comorbidity= 2 if the comorbidity onset occurred within primary observation window
#records with comorbidity=2 will be discarded
T2DM_unique.loc[:,'COPD']=1
T2DM_unique.loc[:,'OA']=1
T2DM_unique.loc[:,'HTN']=1
T2DM_unique.loc[:,'Depression']=1

#put to zero all diagnoses occurred after the end of primary observation window (Encounter date+ 6 months)
T2DM_unique.loc[T2DM_unique['D_COPD_diff']<datetime.timedelta(days=-180), 'COPD'] = 0
T2DM_unique.loc[T2DM_unique['D_OA_diff'] <datetime.timedelta(days=-180), 'OA'] = 0
T2DM_unique.loc[T2DM_unique['D_HTN_diff']<datetime.timedelta(days=-180), 'HTN'] = 0
T2DM_unique.loc[T2DM_unique['D_Depression_diff'] < datetime.timedelta(days=-180), 'Depression'] = 0

# put to 2 all comorbidities appearing during the interval -6months-D_date + 6months ( they will be removed to avoid temporal uncertaintes)
T2DM_unique.loc[(T2DM_unique['D_COPD_diff']>datetime.timedelta(days=-180))&(T2DM_unique['D_COPD_diff']<datetime.timedelta(days=180)), 'COPD'] = 2
T2DM_unique.loc[(T2DM_unique['D_OA_diff'] >datetime.timedelta(days=-180))&(T2DM_unique['D_OA_diff'] <datetime.timedelta(days=180)), 'OA'] = 2
T2DM_unique.loc[(T2DM_unique['D_HTN_diff'] >datetime.timedelta(days=-180))&(T2DM_unique['D_HTN_diff']<datetime.timedelta(days=180)), 'HTN'] = 2
T2DM_unique.loc[(T2DM_unique['D_Depression_diff'] > datetime.timedelta(days=-180))&(T2DM_unique['D_Depression_diff'] < datetime.timedelta(days=180)), 'Depression'] = 2

# put to 0 all comorbidities with NULL on onset date (absence of a diagnosis)
T2DM_unique.loc[T2DM_unique['D_COPD_diff'].isna(), 'COPD'] = 0
T2DM_unique.loc[T2DM_unique['D_OA_diff'].isna(), 'OA'] = 0
T2DM_unique.loc[T2DM_unique['D_HTN_diff'].isna(), 'HTN'] = 0
T2DM_unique.loc[T2DM_unique['D_Depression_diff'].isna(), 'Depression'] = 0

In [ ]:
#remove unnecessary columns (e.g., comorbidity onset dates, encounter date and auxiliary columns)
T2DM_unique_final=T2DM_unique.drop(['D', 'Depression_OnsetDate','HTN_OnsetDate','OA_OnsetDate', 'COPD_Date','T2D_OnsetDate','D_COPD_diff','D_OA_diff', 'D_HTN_diff','D_Depression_diff'], axis=1)
T2DM_unique_final

## Extract the most recent record for patients that will not develop T2D ('T2D'=0)

In [ ]:
healthy=starting_dataset.loc[starting_dataset['T2D']==0]  #non diabetic
healthy

In [ ]:
#check number of unique subjects
healthy['Patient_ID'].nunique()

In [ ]:
#Sort records by EncounterDate (D)
sorted_healthy = healthy.sort_values(by='D',ascending=False)
sorted_healthy

In [ ]:
#select the most recent encounter for each patient
healthy_unique = sorted_healthy.drop_duplicates('Patient_ID', keep='first')
healthy_unique

Once, the most recent encounter for each patient has been extracted, we define the <b>primary observation window</b> as the temporal window centred in 'Encounter Date' (D) +- 6 months

In [ ]:
''' Evaluate presence of comorbidities before the start of the primary observation window to ensure temporal ordering between possible onset
of comorbidities and measured biomarkers.'''
#compute temporal distance between 'Encounter date' and 'comorbidity onset date'
healthy_unique.loc[:,'D_COPD_diff']=healthy_unique.loc[:,'D']-healthy_unique.loc[:,'COPD_Date']
healthy_unique.loc[:,'D_OA_diff']=healthy_unique.loc[:,'D']-healthy_unique.loc[:,'OA_OnsetDate']
healthy_unique.loc[:,'D_HTN_diff']=healthy_unique.loc[:,'D']-healthy_unique.loc[:,'HTN_OnsetDate']
healthy_unique.loc[:,'D_Depression_diff']=healthy_unique.loc[:,'D']-healthy_unique.loc[:,'Depression_OnsetDate']

In [ ]:
#comorbidity =1 if the comorbidity onset occurred before the primary observation window
#comorbidity= 0 if the comorbidity onset never occurred or occurred after the primary observation window
#comorbidity= 2 if the comorbidity onset occurred within primary observation window
#records with comorbidity=2 will be discarded
healthy_unique.loc[:,'COPD']=1
healthy_unique.loc[:,'OA']=1
healthy_unique.loc[:,'HTN']=1
healthy_unique.loc[:,'Depression']=1

#put to zero all diagnoses occurred after the end of primary observation window (Encounter date+ 6 months)healthy_unique.loc[healthy_unique['D_COPD_diff']<datetime.timedelta(days=-180), 'COPD'] = 0
healthy_unique.loc[healthy_unique['D_OA_diff'] <datetime.timedelta(days=-180), 'OA'] = 0
healthy_unique.loc[healthy_unique['D_HTN_diff'] <datetime.timedelta(days=-180), 'HTN'] = 0
healthy_unique.loc[healthy_unique['D_Depression_diff'] < datetime.timedelta(days=-180), 'Depression'] = 0

# put to 2 all comorbidities appearing during the interval -6months-D_date + 6months ( they will be removed to avoid temporal uncertaintes)
healthy_unique.loc[(healthy_unique['D_COPD_diff']<datetime.timedelta(days=180))&(healthy_unique['D_COPD_diff']>datetime.timedelta(days=-180)), 'COPD'] = 2
healthy_unique.loc[(healthy_unique['D_OA_diff'] <datetime.timedelta(days=180))&(healthy_unique['D_OA_diff'] >datetime.timedelta(days=-180)), 'OA'] = 2
healthy_unique.loc[(healthy_unique['D_HTN_diff'] <datetime.timedelta(days=180))&(healthy_unique['D_HTN_diff'] >datetime.timedelta(days=-180)), 'HTN'] = 2
healthy_unique.loc[(healthy_unique['D_Depression_diff'] < datetime.timedelta(days=180))&(healthy_unique['D_Depression_diff'] >datetime.timedelta(days=-180)), 'Depression'] = 2


# put to 0 all comorbidities with NULL on onset date (absence of a diagnosis)
healthy_unique.loc[healthy_unique['COPD_Date'].isna(), 'COPD'] = 0
healthy_unique.loc[healthy_unique['OA_OnsetDate'].isna(), 'OA'] = 0
healthy_unique.loc[healthy_unique['HTN_OnsetDate'].isna(), 'HTN'] = 0
healthy_unique.loc[healthy_unique['Depression_OnsetDate'].isna(), 'Depression'] = 0
#remove unnecessary columns
healthy_unique_final=healthy_unique.drop(['D', 'Depression_OnsetDate','HTN_OnsetDate','OA_OnsetDate', 'COPD_Date','T2D_OnsetDate','D_COPD_diff','D_OA_diff', 'D_HTN_diff','D_Depression_diff'], axis=1)
healthy_unique_final


In [ ]:
#concatenate in one single dataset
processed_dataset=pd.concat([T2DM_unique_final, healthy_unique_final])
processed_dataset

In [ ]:
#eliminate records where comorbidity onset occurs during the observation window
processed_dataset=processed_dataset.loc[(processed_dataset['HTN']!=2)&(processed_dataset['OA']!=2)&(processed_dataset['COPD']!=2)&(processed_dataset['Depression']!=2)]

In [ ]:
processed_dataset

##  Dataset discretization ( according to clinical guidelines)

In [ ]:
processed_dataset=processed_dataset.drop(['Unnamed: 0','Patient_ID'],axis=1)

In [ ]:
processed_dataset['age'] = np.where(processed_dataset['age']<45 , 0, processed_dataset['age'])#Adults
processed_dataset['age'] = np.where((processed_dataset['age']>=45) & (processed_dataset['age']<65), 1, processed_dataset['age'])#middle aged
processed_dataset['age'] = np.where(processed_dataset['age']>=65 , 2, processed_dataset['age'])#older adults

processed_dataset['Total_Cholesterol'] = np.where(processed_dataset['Total_Cholesterol']<5.18 ,0, processed_dataset['Total_Cholesterol']) #desirable
processed_dataset['Total_Cholesterol'] = np.where((processed_dataset['Total_Cholesterol']>=5.18) & (processed_dataset['Total_Cholesterol']<6.19) ,1, processed_dataset['Total_Cholesterol']) #borderline high
processed_dataset['Total_Cholesterol'] = np.where(processed_dataset['Total_Cholesterol']>=6.19 ,2, processed_dataset['Total_Cholesterol']) #high

processed_dataset['FPG'] = np.where(processed_dataset['FPG']<5.6 ,0, processed_dataset['FPG']) #desirable
processed_dataset['FPG'] = np.where((processed_dataset['FPG']>=5.6) & (processed_dataset['FPG']<7) ,1, processed_dataset['FPG']) #borderline high
processed_dataset['FPG'] = np.where(processed_dataset['FPG']>=7 ,2, processed_dataset['FPG']) #high

processed_dataset['TG'] = np.where(processed_dataset['TG']<1.7 ,0, processed_dataset['TG']) #desirable
processed_dataset['TG'] = np.where((processed_dataset['TG']>=1.7) & (processed_dataset['TG']<2.3) ,1, processed_dataset['TG']) #borderline high
processed_dataset['TG'] = np.where((processed_dataset['TG']>=2.3) & (processed_dataset['TG']<5.7) ,2, processed_dataset['TG']) #high
processed_dataset['TG'] = np.where(processed_dataset['TG']>=5.7 ,3, processed_dataset['TG']) #very high

processed_dataset['LDL'] = np.where(processed_dataset['LDL']<2.6 ,0, processed_dataset['LDL']) #optimal
processed_dataset['LDL'] = np.where((processed_dataset['LDL']>=2.6) & (processed_dataset['LDL']<3.4) ,1, processed_dataset['LDL']) #near optimal
processed_dataset['LDL'] = np.where((processed_dataset['LDL']>=3.4) & (processed_dataset['LDL']<4.2) ,2, processed_dataset['LDL']) #borderline high
processed_dataset['LDL'] = np.where((processed_dataset['LDL']>=4.2) & (processed_datasetusal_dataset['LDL']<5) ,3, processed_dataset['LDL']) #high
processed_dataset['LDL'] = np.where(processed_dataset['LDL']>=5 ,4, processed_dataset['LDL']) #very high

processed_dataset['HDL'] = np.where((((processed_dataset['HDL']<1) & (processed_dataset['sex']==1))|((processed_dataset['HDL']<1.3) & (processed_dataset['sex']==0))) ,0, processed_dataset['HDL']) #poor 
processed_dataset['HDL'] = np.where((((processed_dataset['HDL']>=1) &(processed_dataset['HDL']<=1.5) & (processed_dataset['sex']==1))|((processed_dataset['HDL']>=1.3) &(processed_dataset['HDL']<=1.5) & (processed_dataset['sex']==0))) ,1, processed_dataset['HDL']) #better 
processed_dataset['HDL'] = np.where(processed_dataset['HDL']>1.5 ,2, processed_dataset['HDL']) #best

processed_dataset['BMI'] = np.where(processed_dataset['BMI']<18.5 ,0, processed_dataset['BMI']) 
processed_dataset['BMI'] = np.where((processed_dataset['BMI']>=18.5) & (processed_dataset['BMI']<25) ,1, processed_dataset['BMI']) 
processed_dataset['BMI'] = np.where((processed_dataset['BMI']>=25) & (processed_dataset['BMI']<30) ,2, processed_dataset['BMI']) 
processed_dataset['BMI'] = np.where((processed_dataset['BMI']>=30) & (processed_dataset['BMI']<35) ,3, processed_dataset['BMI'])
processed_dataset['BMI'] = np.where((processed_dataset['BMI']>=35) & (processed_dataset['BMI']<40) ,4, processed_dataset['BMI']) 
processed_dataset['BMI'] = np.where(processed_dataset['BMI']>=40 ,5, processed_dataset['BMI'])

#sBP+dBP
processed_dataset['Pressure'] = 0
processed_dataset['Pressure'] = np.where((processed_dataset['sbp']<120) & (processed_dataset['dbp']<80),0, processed_dataset['Pressure']) 
processed_dataset['Pressure'] = np.where(((processed_dataset['sbp']>=120) & (processed_dataset['sbp']<=129)& (processed_dataset['dbp']<80)) ,1, processed_dataset['Pressure']) 
processed_dataset['Pressure'] = np.where((((processed_dataset['sbp']>=130) & (processed_dataset['sbp']<=139))| ((processed_dataset['dbp']>=80)&(processed_dataset['dbp']<=89))) ,2, processed_dataset['Pressure']) 
processed_dataset['Pressure'] = np.where(((processed_dataset['sbp']>=140)| (processed_dataset['dbp']>=90)) ,3, df['Pressure']) 
processed_dataset=processed_dataset.drop(['sbp','dbp'],axis=1)
processed_dataset.describe()

In [ ]:
processed_dataset.to_csv('processed_dataset.csv')
'''save processed dataset: 
-records of the processed dataset without missing values (86618 records) will be used as training set for causal discovery 
of the global causal model
-records with missing values, but complete information about {'age', 'sex', 'sbp', 'dbp', 'BMI','LDL', 'HDL','TG', 'FPG', 'T2D'}
(31864 records) will be used as test set with the aim of deriving an intervention set for counterfactual inference
'''